# Correlation between loop and other modalities

In [1]:
import cooler
import numpy as np
import pandas as pd
from scipy.sparse import triu
from scipy.stats import pearsonr, zscore, norm
from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor, as_completed
from ALLCools.mcds import MCDS
from ALLCools.mcds.utilities import calculate_posterior_mc_frac
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm as cm
import seaborn as sns

mpl.style.use('default')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'


In [2]:
leg = ['L23_IT', 'L4_IT', 'L5_IT', 'L6_IT', 'L6_IT_Car3', 'L56_NP', 'L6_CT', 'L6b', 'L5_ET', 'Amy', 
       'Lamp5', 'Lamp5_LHX6', 'Sncg', 'Vip', 'Pvalb', 'Pvalb_ChC', 'Sst', 'CHD7', 
       'MSN_D1', 'MSN_D2', 'Foxp2', 'SubCtx', 
       'ASC', 'ODC', 'OPC', 'MGC', 'PC', 'EC', 'VLMC'
      ]
legname = ['L2/3-IT', 'L4-IT', 'L5-IT', 'L6-IT', 'L6-IT-Car3', 'L5/6-NP', 'L6-CT', 'L6b', 'L5-ET', 'Amy-Exc', 
       'Lamp5', 'Lamp5-Lhx6', 'Sncg', 'Vip', 'Pvalb', 'Pvalb-ChC', 'Sst', 'Chd7', 
       'MSN-D1', 'MSN-D2', 'Foxp2', 'SubCtx-Cplx', 
       'ASC', 'ODC', 'OPC', 'MGC', 'PC', 'EC', 'VLMC'
      ]
leg2name = {xx:yy for xx,yy in zip(leg, legname)}


In [3]:
leg = {'exc': ['L23_IT', 'L4_IT', 'L5_IT', 'L6_IT', 'L6_IT_Car3', 'L56_NP', 'L6_CT', 'L6b', 'Amy'], 
       'inh': ['Lamp5', 'Lamp5_LHX6', 'Sncg', 'Vip', 'Pvalb', 'Pvalb_ChC', 'Sst', 'CHD7'], 
       'msn': ['MSN_D1', 'MSN_D2', 'Foxp2'], 
       'sub': ['SubCtx'], 
       'glia': ['ASC', 'ODC', 'OPC'], 
       'mgc': ['MGC'], 
       'smc': ['PC'], 
       'endo': ['EC'], 
       'fibro': ['VLMC'],
      }
leg['neu'] = leg['exc'] + leg['inh'] + leg['msn'] + leg['sub']
leg['all'] = leg['neu'] + leg['glia'] + leg['mgc'] + leg['smc'] + leg['endo'] + leg['fibro']


In [4]:
group_name = 'neu'

In [5]:
leg = pd.Index(leg[group_name])
legname = leg.map(leg2name)
res = 10000

In [6]:
indir = '/home/jzhou_salk_edu/sky_workdir/hba/loop_majortype/'
outdir = f'/home/jzhou_salk_edu/sky_workdir/hba/loop_majortype/diff/{group_name}/'

In [7]:
chrom_size_path = f'{indir}hg38_with_chrl.main.chrom.sizes'
chrom_sizes = cooler.read_chromsizes(chrom_size_path, all_names=True)


In [8]:
loopall = pd.read_hdf(f'{outdir}merged_loop.hdf', key='data')
loopall

,0,1,2,3,4,5,Qanova,Eanova,Tanova,mCG_corr,mCH_corr,ATAC_corr
0,chr1,900000,910000,chr1,960000,970000,3.750881,6.097476,2.068213,0.031954,0.094070,0.253426
1,chr1,900000,910000,chr1,970000,980000,3.322128,6.001146,2.007495,-0.090257,0.070470,0.265731
2,chr1,910000,920000,chr1,970000,980000,3.293559,5.439024,2.229271,-0.018746,-0.044665,0.376666
3,chr1,910000,920000,chr1,980000,990000,2.704021,5.648575,2.289167,-0.105304,-0.060823,0.306052
4,chr1,910000,920000,chr1,990000,1000000,2.819877,5.675182,1.669268,-0.354732,-0.320624,0.051738
...,...,...,...,...,...,...,...,...,...,...,...,...
2873610,chr22,50570000,50580000,chr22,50670000,50680000,1.646674,11.822375,1.625390,0.009941,-0.277535,0.078601
2873611,chr22,50580000,50590000,chr22,50670000,50680000,2.256175,11.555016,1.335000,-0.049832,-0.312835,0.082439
2873612,chr22,50590000,50600000,chr22,50670000,50680000,3.531459,11.165133,1.195543,0.075519,-0.135215,0.085896
2873613,chr22,50600000,50610000,chr22,50670000,50680000,4.896728,11.926161,2.028210,0.152859,-0.214300,0.119297


In [9]:
loopq = pd.read_hdf(f'{outdir}loop_Q.hdf', key='data')
loopt = pd.read_hdf(f'{outdir}loop_T.hdf', key='data')


## Load Loop mC

In [11]:
idx1 = loopall[0] + '_' + (loopall[1] // res).astype(str)
idx2 = loopall[3] + '_' + (loopall[4] // res).astype(str)

In [12]:
mcds = MCDS.open('/data/hba/mc_majortype/MajorType.mcds', var_dim='chrom5k')
mcds['chrom10k'] = mcds['chrom5k_chrom'].to_pandas().astype(str) + '_' + (mcds['chrom5k_start'] // 10000).to_pandas().astype(str)
mcds


<xarray.MCDS>
Dimensions:        (cell: 40, chrom5k: 617669, count_type: 2, mc_type: 2)
Coordinates:
  * cell           (cell) <U10 'Amy' 'ASC' 'CA1' 'CA2' ... 'THM_MB' 'Vip' 'VLMC'
  * chrom5k        (chrom5k) object 'chr1_0' 'chr1_1' ... 'chrY_11445'
    chrom5k_chrom  (chrom5k) <U5 'chr1' 'chr1' 'chr1' ... 'chrY' 'chrY' 'chrY'
    chrom5k_end    (chrom5k) int64 5000 10000 15000 ... 57225000 57227415
    chrom5k_start  (chrom5k) int64 0 5000 10000 ... 57215000 57220000 57225000
  * count_type     (count_type) <U3 'mc' 'cov'
  * mc_type        (mc_type) <U3 'CGN' 'CHN'
Data variables:
    chrom5k_da     (cell, chrom5k, mc_type, count_type) uint32 dask.array<chunksize=(5, 77209, 1, 1), meta=np.ndarray>
    chrom10k       (chrom5k) object 'chr1_0' 'chr1_0' ... 'chrY_5722'
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

### mCG

In [13]:
mc = mcds['chrom5k_da'].sel(count_type='mc', mc_type='CGN').to_pandas().T
mc['chrom10k'] = mcds['chrom10k'].to_pandas()
mc = mc.groupby('chrom10k').sum().T
cov = mcds['chrom5k_da'].sel(count_type='cov', mc_type='CGN').to_pandas().T
cov['chrom10k'] = mcds['chrom10k'].to_pandas()
cov = cov.groupby('chrom10k').sum().T


In [14]:
binfilter = ['_'.join(xx.split('_')[:-1]) for xx in mc.columns]
binfilter = [(len(xx)<6) and (xx not in ['chrM','chrX','chrY']) for xx in binfilter]
print(np.sum(binfilter))
mc = mc.loc[leg, binfilter]
cov = cov.loc[leg, binfilter]
print(mc.shape, cov.shape)


287509
(21, 287509) (21, 287509)


In [15]:
ratio = calculate_posterior_mc_frac(mc.values, cov.values)
ratio = pd.DataFrame(ratio, index=leg, columns=mc.columns)
ratio


chrom10k,chr10_0,chr10_1,chr10_10,chr10_100,chr10_1000,chr10_10000,chr10_10001,chr10_10002,chr10_10003,chr10_10004,...,chr9_9990,chr9_9991,chr9_9992,chr9_9993,chr9_9994,chr9_9995,chr9_9996,chr9_9997,chr9_9998,chr9_9999
L23_IT,1.0,0.865750,1.098113,1.104957,0.970369,0.546823,0.765560,1.030258,1.122590,1.044473,...,0.444581,1.120092,1.100522,1.093261,1.098141,1.069337,1.101062,1.070954,1.024206,0.910354
L4_IT,1.0,0.829846,1.079987,1.100184,1.031948,0.544228,0.771079,1.003114,1.103790,1.051402,...,0.420759,1.109077,1.098732,1.061285,1.072169,1.095959,1.089031,1.092414,1.054224,0.718477
L5_IT,1.0,0.871156,1.086426,1.090221,0.994796,0.572899,0.733758,0.978924,1.100353,1.021597,...,0.448407,1.106363,1.069185,1.078171,1.086915,1.044753,1.098709,1.076597,1.038894,1.003211
L6_IT,1.0,0.841410,1.096542,1.104306,1.026670,0.584123,0.744700,1.045663,1.108378,1.030944,...,0.431908,1.115438,1.104424,1.086339,1.107761,1.018649,1.108945,1.097433,1.064725,0.993141
L6_IT_Car3,1.0,0.873949,1.090812,1.096528,1.091477,0.534639,0.701619,1.057742,1.096929,1.004430,...,0.426434,1.112644,1.107946,1.083099,1.102255,1.067188,1.098333,1.078087,1.071479,0.834742
L56_NP,1.0,0.797610,1.057855,1.061449,1.023070,0.559595,0.698354,0.937548,1.072523,1.008295,...,0.423151,1.075689,1.046400,1.038178,1.058969,1.048859,1.047916,1.030193,1.039942,1.069534
L6_CT,1.0,0.852024,1.091768,1.099437,1.044037,0.528753,0.785238,0.993313,1.105819,1.042183,...,0.424429,1.108913,1.100481,1.087009,1.105778,0.998863,1.079483,1.073137,1.042358,1.088393
L6b,1.0,0.878122,1.102235,1.096597,0.910365,0.614620,0.712343,1.069397,1.109824,1.016281,...,0.457184,1.109582,1.114130,1.063698,1.099670,0.983586,1.092634,1.062037,1.012311,0.950932
Amy,1.0,0.851257,1.097555,1.107771,0.990718,0.491762,0.724681,1.016296,1.116206,1.050156,...,0.442131,1.127280,1.097345,1.108722,1.102675,1.047166,1.113539,1.068260,1.089860,1.074183
Lamp5,1.0,0.805150,1.033116,1.088847,1.096238,0.512818,0.724184,1.062630,1.091241,1.021778,...,0.434840,1.096492,1.069307,1.090170,1.084815,1.073408,1.077023,1.072035,1.080043,1.079055


In [16]:
loopcg = (ratio[idx1].values + ratio[idx2].values).T / 2
loopcg = pd.DataFrame(loopcg, index=loopq.index, columns=leg)


In [17]:
loopall['mCG_corr'] = [pearsonr(xx,yy)[0] for xx,yy in zip(loopcg.values, loopq.values)]


### mCH

In [18]:
mc = mcds['chrom5k_da'].sel(count_type='mc', mc_type='CHN').to_pandas().T
mc['chrom10k'] = mcds['chrom10k'].to_pandas()
mc = mc.groupby('chrom10k').sum().T
cov = mcds['chrom5k_da'].sel(count_type='cov', mc_type='CHN').to_pandas().T
cov['chrom10k'] = mcds['chrom10k'].to_pandas()
cov = cov.groupby('chrom10k').sum().T


In [19]:
mc = mc.loc[leg, binfilter]
cov = cov.loc[leg, binfilter]
print(mc.shape, cov.shape)


(21, 287509) (21, 287509)


In [20]:
ratio = calculate_posterior_mc_frac(mc.values, cov.values)
ratio = pd.DataFrame(ratio, index=leg, columns=mc.columns)
ratio


chrom10k,chr10_0,chr10_1,chr10_10,chr10_100,chr10_1000,chr10_10000,chr10_10001,chr10_10002,chr10_10003,chr10_10004,...,chr9_9990,chr9_9991,chr9_9992,chr9_9993,chr9_9994,chr9_9995,chr9_9996,chr9_9997,chr9_9998,chr9_9999
L23_IT,1.0,0.581408,1.048868,0.242306,0.609380,0.868303,1.127298,1.081475,1.683425,1.620382,...,0.759118,0.580684,0.614478,0.674427,0.580364,0.553518,0.636553,0.463157,0.426187,0.392909
L4_IT,1.0,0.495013,0.795514,0.245440,0.963888,1.094922,1.168507,1.088782,1.546328,1.765103,...,0.846806,0.786174,0.756176,0.789065,0.802313,0.947783,0.883354,0.705005,0.579375,0.484728
L5_IT,1.0,0.455561,1.015350,0.228945,0.840501,1.270735,1.163866,1.093905,1.705293,1.773222,...,0.864993,0.658063,0.572586,0.707412,0.744451,0.576037,0.699209,0.611104,0.495135,0.455101
L6_IT,1.0,0.650953,1.057721,0.240116,0.898931,1.362157,1.445102,1.237765,1.786124,1.641940,...,0.741295,0.624843,0.609334,0.696892,0.714934,0.526063,0.674852,0.523097,0.412630,0.391960
L6_IT_Car3,1.0,0.575051,1.077855,0.224761,1.325802,0.861500,1.221765,1.217034,1.706987,1.594311,...,0.756624,0.621904,0.665758,0.670322,0.707093,0.612832,0.649896,0.507689,0.448870,0.378274
L56_NP,1.0,0.402756,0.881177,0.187907,0.880132,1.413384,1.596061,1.131252,2.004234,2.065186,...,0.636043,0.541412,0.470409,0.574378,0.622158,0.716971,0.701500,0.593755,0.432142,0.351776
L6_CT,1.0,0.444559,0.978495,0.277735,1.018575,1.440508,1.389699,1.128223,1.888419,1.825062,...,0.643229,0.474623,0.578430,0.663219,0.629395,0.454445,0.595501,0.479590,0.393849,0.387964
L6b,1.0,0.445212,1.214124,0.392053,0.693349,1.660831,1.387080,1.277147,1.901074,1.448106,...,0.657112,0.503805,0.520558,0.580309,0.642889,0.478172,0.602957,0.457603,0.336922,0.341128
Amy,1.0,0.516281,1.099986,0.236326,0.984687,0.700906,1.150781,0.997126,1.717197,1.658379,...,0.732941,0.549465,0.526880,0.701293,0.601888,0.488402,0.583821,0.471135,0.456757,0.394323
Lamp5,1.0,0.396334,0.570967,0.270190,1.144927,0.668095,1.072014,1.080740,1.521001,1.666255,...,0.675580,0.596514,0.467924,0.610111,0.579131,0.587864,0.589750,0.568379,0.550557,0.388556


In [21]:
loopch = (ratio[idx1].values + ratio[idx2].values).T / 2
loopch = pd.DataFrame(loopch, index=loopq.index, columns=leg)


In [22]:
loopall['mCH_corr'] = [pearsonr(xx,yy)[0] for xx,yy in zip(loopch.values, loopq.values)]


### ATAC

In [23]:
sig = pd.read_hdf('/home/jzhou_salk_edu/sky_workdir/hba/atac_majortype/cluster_atac_signal.hdf')
cov = pd.read_hdf('/home/jzhou_salk_edu/sky_workdir/hba/atac_majortype/cluster_atac_cov.hdf')

In [24]:
bins = pd.DataFrame(index=sig.columns)
bins['chrom'] = bins.index.str.split('_').str[0]
bins['start'] = (bins.index.str.split('_').str[1].astype(int) - 1) * 5000
bins['chrom10k'] = bins['chrom'] + '_' + (bins['start'] // res).astype(str)

In [25]:
sig = sig.groupby(by=bins['chrom10k'], axis=1).sum()
cov = cov.groupby(by=bins['chrom10k']).sum()
atac = (sig/cov).fillna(0)

In [26]:
legatac = leg[leg.isin(atac.index)]
atac = atac.loc[legatac]
atac = atac / atac.sum(axis=1)[:, None]

In [27]:
loopatac = (atac[idx1].values + atac[idx2].values).T / 2
loopatac = pd.DataFrame(loopatac, index=loopq.index, columns=legatac)


In [28]:
loopall['ATAC_corr'] = [pearsonr(xx,yy)[0] for xx,yy in zip(loopatac[legatac].values, loopq[legatac].values)]


In [29]:
loopall


,0,1,2,3,4,5,Qanova,Eanova,Tanova,mCG_corr,mCH_corr,ATAC_corr
0,chr1,900000,910000,chr1,960000,970000,3.750881,6.097476,2.068213,0.031954,0.094070,0.253426
1,chr1,900000,910000,chr1,970000,980000,3.322128,6.001146,2.007495,-0.090257,0.070470,0.265731
2,chr1,910000,920000,chr1,970000,980000,3.293559,5.439024,2.229271,-0.018746,-0.044665,0.376666
3,chr1,910000,920000,chr1,980000,990000,2.704021,5.648575,2.289167,-0.105304,-0.060823,0.306052
4,chr1,910000,920000,chr1,990000,1000000,2.819877,5.675182,1.669268,-0.354732,-0.320624,0.051738
...,...,...,...,...,...,...,...,...,...,...,...,...
2873610,chr22,50570000,50580000,chr22,50670000,50680000,1.646674,11.822375,1.625390,0.009941,-0.277535,0.078601
2873611,chr22,50580000,50590000,chr22,50670000,50680000,2.256175,11.555016,1.335000,-0.049832,-0.312835,0.082439
2873612,chr22,50590000,50600000,chr22,50670000,50680000,3.531459,11.165133,1.195543,0.075519,-0.135215,0.085896
2873613,chr22,50600000,50610000,chr22,50670000,50680000,4.896728,11.926161,2.028210,0.152859,-0.214300,0.119297


In [30]:
loopall.to_hdf(f'{outdir}merged_loop.hdf', key='data')
loopcg.to_hdf(f'{outdir}loop_mCG.hdf', key='data')
loopch.to_hdf(f'{outdir}loop_mCH.hdf', key='data')
loopatac.to_hdf(f'{outdir}loop_ATAC.hdf', key='data')
